In [1]:
%matplotlib inline 
import numpy as np
import pandas as pd

import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

from utils import rstr

In [2]:
df =pd.read_csv('data/reviews_metadata.csv',parse_dates=['date'])
df_listing = pd.read_csv('data/listings.csv')

In [3]:
df.date.max()

Timestamp('2017-05-08 00:00:00')

In [4]:
df.head()

,listing_id,date
0,7878281,2015-09-12
1,7878281,2015-10-09
2,7878281,2015-11-15
3,7878281,2016-01-02
4,7878281,2016-04-28


In [5]:
last_scraped=pd.to_datetime(df_listing.last_scraped).max().date()
last_scraped

datetime.date(2017, 5, 8)

In [6]:
df['from_scraping_date']=(last_scraped-df.date).dt.days

In [7]:
bins = [0,7,14,20,30,40,50,60,70,90,100,2000]
group_names = ["reviewed_more_than_"+str(i)+"_ago" for i in bins[:-1]]

In [8]:
categories = pd.cut(df['from_scraping_date'], bins,labels=group_names)
df['review_age']=categories

In [9]:
df.from_scraping_date.head()

0    604
1    577
2    540
3    492
4    375
Name: from_scraping_date, dtype: int64

In [10]:
df_group_review_age = df.groupby(['listing_id','review_age'])['date'].count()
df_group_review_age= df_group_review_age.unstack('review_age').fillna(0)

In [11]:
df_group_review_age.to_csv('data/reviews_frequency.csv')

In [12]:
df_test=pd.read_csv('data/reviews_frequency.csv')

In [13]:
df_listing = df_listing[(df_listing.reviews_per_month>0)&(df_listing.availability_30>0)]

In [14]:
df_test = pd.merge(df_listing[['id','availability_30']],df_test,left_on='id',right_on='listing_id')

In [15]:
df_test.drop(['id','listing_id'],axis=1,inplace=True)

In [19]:
df_test.corr()[['availability_30']].sort_values(by='availability_30',ascending=True)

,availability_30
reviewed_more_than_40_ago,-0.199318
reviewed_more_than_30_ago,-0.193696
reviewed_more_than_20_ago,-0.187662
reviewed_more_than_60_ago,-0.177572
reviewed_more_than_14_ago,-0.168346
reviewed_more_than_70_ago,-0.164634
reviewed_more_than_90_ago,-0.163673
reviewed_more_than_100_ago,-0.142606
reviewed_more_than_50_ago,-0.138040
reviewed_more_than_7_ago,-0.116951


In [17]:
df_group_review_age.index

Int64Index([    1971,     2015,     3176,     3309,     9991,    11203,
               16644,    17409,    17904,    20858,
            ...
            18452548, 18453841, 18458311, 18461865, 18462323, 18472802,
            18512476, 18523824, 18546940, 18547011],
           dtype='int64', name=u'listing_id', length=16174)

In [18]:
df[df.listing_id==2015]

,listing_id,date,from_scraping_date,review_age
142023,2015,2016-04-11,392,reviewed_more_than_100_ago
142024,2015,2016-04-15,388,reviewed_more_than_100_ago
142025,2015,2016-04-26,377,reviewed_more_than_100_ago
142026,2015,2016-05-10,363,reviewed_more_than_100_ago
142027,2015,2016-05-14,359,reviewed_more_than_100_ago
142028,2015,2016-05-16,357,reviewed_more_than_100_ago
142029,2015,2016-05-28,345,reviewed_more_than_100_ago
142030,2015,2016-05-31,342,reviewed_more_than_100_ago
142031,2015,2016-06-03,339,reviewed_more_than_100_ago
142032,2015,2016-06-27,315,reviewed_more_than_100_ago
